In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName("sparkbyexamples.com").getOrCreate()

24/06/09 11:29:47 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/06/09 11:29:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/09 11:29:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/09 11:29:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/09 11:29:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
emp = ([
    (1,1,"Smith",-1,"2018","10","M",3000),
    (2,2,"Rose",1,"2010","20","M",4000),
    (3,3,"Williams",1,"2010","10","M",1000),
    (4,4,"Jones",2,"2005","10","F",2000),
  ])
empColumns = (["surrogate_key","emp_id","name","superior_emp_id","year_joined",
       "emp_dept_id","gender","salary"])

oldDF = spark.createDataFrame(data=emp, schema = empColumns)
# oldDF.printSchema()
print('oldDF')
oldDF.show(truncate=False)

emp_new = ([
    (1,"Smith",-1,"2018","10","M",3000),
    (2,"Rose",1,"2010","20","M",4000),
    (3,"Williams",1,"2010","10","M",1000),
    (4,"Jones",2,"2005","10","F",2000),
    (5,"Brown",2,"2010","40","",5000),
    (6,"Brown",2,"2010","50","",3500)
  ])
emp_newColumns = (["emp_id","name","superior_emp_id","year_joined",
       "emp_dept_id","gender","salary"])
newDF = spark.createDataFrame(data=emp_new, schema = emp_newColumns)
# newDF.printSchema()
print('emp_new')
newDF.show(truncate=False)

oldDF


+-------------+------+--------+---------------+-----------+-----------+------+------+
|surrogate_key|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+-------------+------+--------+---------------+-----------+-----------+------+------+
|1            |1     |Smith   |-1             |2018       |10         |M     |3000  |
|2            |2     |Rose    |1              |2010       |20         |M     |4000  |
|3            |3     |Williams|1              |2010       |10         |M     |1000  |
|4            |4     |Jones   |2              |2005       |10         |F     |2000  |
+-------------+------+--------+---------------+-----------+-----------+------+------+

emp_new
+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose   

In [3]:
# SELECIONAR DADOS NOVOS
newData = (newDF.join(oldDF
            ,((oldDF.emp_id == newDF.emp_id) &
                (oldDF.name == newDF.name) &
                (oldDF.superior_emp_id == newDF.superior_emp_id) &
                (oldDF.year_joined == newDF.year_joined) &
                (oldDF.emp_dept_id == newDF.emp_dept_id) &
                (oldDF.gender == newDF.gender) &
                (oldDF.salary == newDF.salary))
            ,"leftanti"))
newData.show(truncate=False)

+------+-----+---------------+-----------+-----------+------+------+
|emp_id|name |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|5     |Brown|2              |2010       |40         |      |5000  |
|6     |Brown|2              |2010       |50         |      |3500  |
+------+-----+---------------+-----------+-----------+------+------+



In [4]:
# ENCONTRAR ULTIMA SURROGATE KEY
maxSK = oldDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
maxSK

4

In [5]:
# ADICIONAR SK AO NOVO DF
windowSpec = Window.orderBy("emp_id")

deptDF_SK = (newData.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
deptDF_SK.show()

24/06/08 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 1

+------+-----+---------------+-----------+-----------+------+------+-------------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|surrogate_key|
+------+-----+---------------+-----------+-----------+------+------+-------------+
|     5|Brown|              2|       2010|         40|      |  5000|            5|
|     6|Brown|              2|       2010|         50|      |  3500|            6|
+------+-----+---------------+-----------+-----------+------+------+-------------+



In [6]:
# FINAL DO PROCESSO
finalDF = oldDF.unionByName(deptDF_SK)
finalDF.show()

24/06/08 11:54:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 11:54:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/08 1

+-------------+------+--------+---------------+-----------+-----------+------+------+
|surrogate_key|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+-------------+------+--------+---------------+-----------+-----------+------+------+
|            1|     1|   Smith|             -1|       2018|         10|     M|  3000|
|            2|     2|    Rose|              1|       2010|         20|     M|  4000|
|            3|     3|Williams|              1|       2010|         10|     M|  1000|
|            4|     4|   Jones|              2|       2005|         10|     F|  2000|
|            5|     5|   Brown|              2|       2010|         40|      |  5000|
|            6|     6|   Brown|              2|       2010|         50|      |  3500|
+-------------+------+--------+---------------+-----------+-----------+------+------+



INNER JOIN

In [18]:
clientes = ([
    (1,"Smith",),
    (2,"Rose"),
    (3,"Williams"),
    (4,"Jones"),
    (4,"Charles"),
  ])
clientesColumns = (["id_cliente","name"])

clientesDF = spark.createDataFrame(data=clientes, schema = clientesColumns)
# clientesDF.printSchema()
print('clientesDF')
clientesDF.show(truncate=False)

vendas = ([
    (1, 4),
    (2, 3),
    (3, 2),
    (4, 1)
  ])
vendasColumns = (["id","id_cliente"])
vendasDF = spark.createDataFrame(data=vendas, schema = vendasColumns)
# vendasDF.printSchema()
print('vendasDF')
vendasDF.show(truncate=False)

clientesDF
+----------+--------+
|id_cliente|name    |
+----------+--------+
|1         |Smith   |
|2         |Rose    |
|3         |Williams|
|4         |Jones   |
|4         |Charles |
+----------+--------+

vendasDF
+---+----------+
|id |id_cliente|
+---+----------+
|1  |4         |
|2  |3         |
|3  |2         |
|4  |1         |
+---+----------+



In [19]:
joinDF = vendasDF.join(clientesDF, ['id_cliente']
                       # , 'fullouter'
                       , 'inner'
                      )
joinDF.show()

+----------+---+--------+
|id_cliente| id|    name|
+----------+---+--------+
|         1|  4|   Smith|
|         2|  3|    Rose|
|         3|  2|Williams|
|         4|  1|   Jones|
|         4|  1| Charles|
+----------+---+--------+

